In [328]:
import pandas as pd
import numpy as np

data = pd.read_csv('forwardkeys-Arrivals-BookingsHistory-_PAR_.csv')
data.head(3)

,Pax,PaxPerBooking,BookingDate,TravelDate,ArrivalHour,DepartureHour,LengthOfTrip,TripOrigin,NumInboundGateways,PointOfInterest,LengthOfStayAtPOI,TripHighestCabin,PaxProfile,DistChannel,NumFurtherDestiations
0,1,1,141113,141205,17,19,8,JP,2,CDG_2G,1,F,B,R,6
1,-1,1,141112,150115,12,18,6,JP,4,CDG_2G,1,T,B,R,1
2,-1,1,141113,150115,12,18,6,JP,2,CDG_2G,1,T,B,R,1


In [329]:
data.dtypes

Pax                       int64
PaxPerBooking             int64
BookingDate               int64
TravelDate                int64
ArrivalHour               int64
DepartureHour             int64
LengthOfTrip             object
TripOrigin               object
NumInboundGateways        int64
PointOfInterest          object
LengthOfStayAtPOI        object
TripHighestCabin         object
PaxProfile               object
DistChannel              object
NumFurtherDestiations     int64
dtype: object

## Data types and pre-processing

At first glance we find some issues:
- The format of date columns within the dataframe. They appear to be integers instead of timestamps. In order to approach date values, we could **change both 'BookingDate' and 'TravelDate' into a date type record** as shown below
- 'LengthOfTrip' and 'LengthOfStayAtPOI' appear to be 'object' but the should be numbers as they refe to numerical variables. Let´s also change their values just in case we need to make some operations with them along the analysis.
- 'Pax' shows the same information than 'PaxPerBooking', being turned into negative just to show **whether a booking has been cancelled or not**. let´s change it´s format to binary and rename the column to "Cancelled". 

In [330]:
# Date columns
data['BookingDate'] = pd.to_datetime(data['BookingDate'], format='%y%m%d', errors='ignore')
data['TravelDate'] = pd.to_datetime(data['TravelDate'], format='%y%m%d', errors='ignore')

# Num columns
cols_to_change = []
for col in cols_to_change:
    data[col] = pd.to_numeric(data[col], errors = 'coerce', downcast = 'integer')

# Cancellations     
data['Pax'] = data['Pax'].apply(lambda x: 1 if x < 0 else 0)
data.rename(columns={"Pax" : "Cancelled"}, inplace=True)

# Check new df
data.head(3)

,Cancelled,PaxPerBooking,BookingDate,TravelDate,ArrivalHour,DepartureHour,LengthOfTrip,TripOrigin,NumInboundGateways,PointOfInterest,LengthOfStayAtPOI,TripHighestCabin,PaxProfile,DistChannel,NumFurtherDestiations
0,0,1,2014-11-13,2014-12-05,17,19,8,JP,2,CDG_2G,1,F,B,R,6
1,1,1,2014-11-12,2015-01-15,12,18,6,JP,4,CDG_2G,1,T,B,R,1
2,1,1,2014-11-13,2015-01-15,12,18,6,JP,2,CDG_2G,1,T,B,R,1



## Questions to answer

1. How many Chinese booked between Nov 16th and Nov 28th 2015 for travelling between Dec 25 and Dec 31?
2. What useful finding would you stress in an executive summary?

Our first approach will be **filtering the dataframe for those travellers booking on the selected dates for the target dates**. We will find a number of net bookings, but we´ll then face the issue of not having anything to compare with. That´s why it could make sense to **make the same analysis for the same dates of 2014**. For efficiency purposes, let´s define a function to perform the analysis, pass the different years to it and finally show the results.

In [1]:
def analysis(df, year):
    filtered_df= df[(df['BookingDate'] >= year+'-11-16') & 
                         (df['BookingDate'] <= year+'-11-28') &
                         (df['TravelDate'] >= year+'-12-25') &
                         (df['TravelDate'] <= year+'-12-31')].sort_values('BookingDate')
    cancel_df = filtered_df[filtered_df['Cancelled'] == 1]
    net_bookings = filtered_df.shape[0] - cancel_df.shape[0]
    cancel_ratio = cancel_df.shape[0] / filtered_df.shape[0]
    return filtered_df, cancel_df, net_bookings, cancel_ratio

In [357]:
chinese_data = data[data['TripOrigin']=='CN']

chinese_filtered_data_15, chinese_cancel_df_15, chinese_net_bookings_15, chinese_cancel_ratio_15 = analysis(chinese_data, '2015')
chinese_filtered_data_14, chinese_cancel_df_14, chinese_net_bookings_14, chinese_cancel_ratio_14 = analysis(chinese_data, '2014')

print('Chinese booking between 16-28 Nov 2015 for travelling between 25-31 Dic:')
print('\n')
print('Year 2014, Net bookings: {}, Cancellations: {}, Total bookings: {}, Cancelling ratio: {}'.
      format(chinese_net_bookings_14 , chinese_cancel_df_14.shape[0] , chinese_filtered_data_14.shape[0], round(chinese_cancel_ratio_14,2)))
print('Year 2015, Net bookings: {}, Cancellations: {}, Total bookings: {}, Cancelling ratio: {}'.
      format(chinese_net_bookings_15 , chinese_cancel_df_15.shape[0] , chinese_filtered_data_15.shape[0], round(chinese_cancel_ratio_15,2)))
print('Net bookings variation: {}, cancelling ratio variation: {}'.
      format(chinese_net_bookings_15 - chinese_net_bookings_14, round(chinese_cancel_ratio_15 - chinese_cancel_ratio_14, 2)))

Chinese booking between 16-28 Nov 2015 for travelling between 25-31 Dic:


Year 2014, Net bookings: 397, Cancellations: 262, Total bookings: 659, Cancelling ratio: 0.4
Year 2015, Net bookings: 264, Cancellations: 227, Total bookings: 491, Cancelling ratio: 0.46
Net bookings variation: -133, cancelling ratio variation: 0.06


In [354]:
filtered_data_15, cancel_df_15, net_bookings_15, cancel_ratio_15 = analysis(data, '2015')
filtered_data_14, cancel_df_14, net_bookings_14, cancel_ratio_14 = analysis(data, '2014')

print('Travellers booking between 16-28 Nov 2015 for travelling between 25-31 Dic:')
print('\n')
print('Year 2014, Net bookings: {}, Cancellations: {}, Total bookings: {}, Cancelling ratio: {}'.
      format(net_bookings_14 , cancel_df_14.shape[0] , filtered_data_14.shape[0], round(cancel_ratio_14,2)))
print('Year 2015, Net bookings: {}, Cancellations: {}, Total bookings: {}, Cancelling ratio: {}'.
      format(net_bookings_15 , cancel_df_15.shape[0] , filtered_data_15.shape[0], round(cancel_ratio_15,2)))
print('Net bookings variation: {}, cancelling ratio variation: {}'.
      format(net_bookings_15 - net_bookings_14, round(cancel_ratio_15 - cancel_ratio_14, 2)))

Travellers booking between 16-28 Nov 2015 for travelling between 25-31 Dic:


Year 2014, Net bookings: 3597, Cancellations: 2780, Total bookings: 6377, Cancelling ratio: 0.44
Year 2015, Net bookings: 1997, Cancellations: 2690, Total bookings: 4687, Cancelling ratio: 0.57
Net bookings variation: -1600, cancelling ratio variation: 0.14


## For all the countries: 
- It seems that **net bookings dropped dramatically in 2015**, with 1600 less bookings compared to 2014. However we should look into proportional metrics rather than absolut ones to have a real picture of the fact. 
- When looking at the **variation of the cancelling ratio we find that it raises in 13 percentage points**. 57% of travellers booked their tickets and cancelled them around those dates. Regarding the potential increase of tourism heading Europe (specially Paris, one of the most visited cities in the world), **it is clear that something happened during those dates.**

A quick search in google shows us a **remarkable event that shocked the world on 13th Nov 2015: the terrorist attacks in Paris. Three days later, on Nov 16th, Françoise Hollande declared France was at war** after the acts of war performed against the country by islamic terrorists. **It clearly pushed people to cancel their bookings to Paris**.


## For China:

It seems that net bookings decreased by 2015, **with a total of 264 bookings**. It means 133 less bookings compared to 2014. The variation of the cancellation changes in 6 percentage points. 46% of travellers booked their tickets and cancelled them, not very different than the previous year.

The dataset contains mainly countries from Asia and Oceania. Despite the huge inffluence of the event occured in Paris for all the countries contained in the dataset, **China seems to be less influenced**. 

### **Why?**

As an hypothesis we could think that **the control of news and information performed by the chinese government could made users to be less aware of the supposed risk of travelling to France.** We can check in some cells below that other countries with a wider access to information as Japan and Korea raised their cancellation ratio after the news were spreaded.

## Let´s go deeper into the analysis with other countries.

##### **What kind of people cancelled their bookings?** We have much more information housed in our dataset. Let´s explore it.


In [281]:
# Define a function to group the 2015 filtered dataset by different features, to try to find
# patterns and difference between different traveller profiles

def analysis_group_by(feature, df):
    # Check total bookings, cancellations and cancellation ratio per  airport
    group_obj = df.groupby(feature)
    cancellations_by = group_obj['Cancelled'].sum()
    total_bookings_by = group_obj['Cancelled'].count()

    # Store it in a DataFrame
    index = cancellations_by.index
    info = {'cancelled': cancellations_by, 
            'total': total_bookings_by,
            'cancel_ratio': round(cancellations_by / total_bookings_by, 2)}

    df_by = pd.DataFrame(info, index = index, columns = ['cancelled', 'total', 'cancel_ratio']).sort_values('cancel_ratio', ascending=False)
    
    return df_by

In [340]:
df_by_Country_15 = analysis_group_by('TripOrigin',filtered_data_15).sort_values('TripOrigin')
df_by_Country_14 = analysis_group_by('TripOrigin',filtered_data_14).sort_values('TripOrigin')

df_by_Country_15['cancel_ratio_14'] = df_by_Country_14['cancel_ratio']
df_by_Country_15['cancel_ratio_variation'] = df_by_Country_15['cancel_ratio'] - df_by_Country_15['cancel_ratio_14'] 


In [343]:
df_by_Country_15.sort_values('cancel_ratio_variation', ascending = False)

,cancelled,total,cancel_ratio,cancel_ratio_14,cancel_ratio_variation
TripOrigin,,,,,
NP,2,3,0.67,0.00,0.67
MM,7,11,0.64,0.33,0.31
JP,702,958,0.73,0.46,0.27
KR,467,734,0.64,0.43,0.21
LA,1,5,0.20,0.00,0.20
VN,40,85,0.47,0.30,0.17
PK,30,49,0.61,0.47,0.14
TW,63,115,0.55,0.43,0.12
TH,154,278,0.55,0.43,0.12


- When comparing countries, and just consideing those with a remarkable amount of total bookings, Japan and South Korea appear to be the ones with the higher cancellation ratio variation compared to the previous year (27% and 21% increase respectively). As two of the richest economies in Asia, it makes sense they could be more aware of threatening news.

In [284]:
df_by_PaxProfile = analysis_group_by('PaxProfile',filtered_data_15)
df_by_PaxProfile.index = ['Leisure', 'Group', 'Bussiness', 'Visit']
df_by_PaxProfile

,cancelled,total,cancel_ratio
Leisure,1249,2028,0.62
Group,103,169,0.61
Bussiness,1105,1981,0.56
Visit,233,509,0.46


In [279]:
df_by_PaxPerBooking = analysis_group_by('PaxPerBooking',filtered_data_15)
df_by_PaxPerBooking

,cancelled,total,cancel_ratio
PaxPerBooking,,,
7,34,45,0.76
2,911,1416,0.64
3,197,324,0.61
4,196,320,0.61
6,82,142,0.58
5,46,84,0.55
1,1224,2356,0.52


- It makes sense that people **people travelling for holidays (leisure and group profiles) are more prone to cancel than those heading Paris for bussiness (they have to do it) or visiting friends/family** (Fear makes people to come closer to loved ones, no matter the risk)


- This relates with the fact that bookings including more people were cancelled in a higher ratio than those bookings made for single travellers. People travelling for bussinesses use to travel alone, similarly to those doing it for visiting family or friends. The next cell shows it clearly. 86% of Bussiness and 80% of Visit labeled bookings were made for single travellers, whereas just 11% of leisure travellers were booking to fly alone. We excluded group labeled bookings for obvious reasons. 

In [312]:
buss = (filtered_data_15[filtered_data_15['PaxProfile'] == 'B']['PaxPerBooking'].value_counts() / df_by_PaxProfile['total'][2]).sort_values()
visit = (filtered_data_15[filtered_data_15['PaxProfile'] == 'V']['PaxPerBooking'].value_counts() / df_by_PaxProfile['total'][3]).sort_values()
leis = (filtered_data_15[filtered_data_15['PaxProfile'] == 'L']['PaxPerBooking'].value_counts() / df_by_PaxProfile['total'][0]).sort_values()

print('Bussiness', buss, 'Visit', visit, 'Leisure', leis)


Bussiness 7    0.002019
5    0.004543
6    0.007067
4    0.012620
3    0.019182
2    0.092378
1    0.862191
Name: PaxPerBooking, dtype: float64 Visit 5    0.003929
3    0.025540
4    0.031434
2    0.135560
1    0.803536
Name: PaxPerBooking, dtype: float64 Leisure 5    0.035996
1    0.117850
3    0.134615
4    0.137574
2    0.573964
Name: PaxPerBooking, dtype: float64


In [271]:
df_by_DistChannel = analysis_group_by('DistChannel',filtered_data_15)
df_by_DistChannel

,cancelled,total,cancel_ratio
DistChannel,,,
S,281,403,0.70
R,1776,3126,0.57
C,250,451,0.55
W,383,707,0.54


- Regarding the different channels the bookings were made through, the cancelling ratio is much higher for those bookings not labeled as Retail, Corporate or Online travel agencys. We could suppose those bookings were made directly by the users with the airline or flight search engine, so that they were able to cancel it easier than those made through any kind of travel agency.